In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd/content/drive/My Drive/딥러닝 실습/5.4.Sentiment Analysis

/content/drive/My Drive/딥러닝 실습/5.4.Sentiment Analysis


In [3]:
import pandas as pd
from model.LSTM import LSTM_model
from preprocessing.text_preprocessing import Sentences2LemmaWords
from preprocessing.text_preprocessing import GetUnique
from preprocessing.text_preprocessing import setTokenizer
from preprocessing.text_preprocessing import LemmaWords2Seqeunces
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import time
from log.logger import Logger

Using TensorFlow backend.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
train = pd.read_csv('/content/drive/My Drive/딥러닝 실습/5.4.Sentiment Analysis/5.4.data/train.tsv', sep='\t')
test = pd.read_csv('/content/drive/My Drive/딥러닝 실습/5.4.Sentiment Analysis/5.4.data/test.tsv', sep='\t')

print('train data shape: ', train.shape)
print('test data shape: ', test.shape)
print('----------[train infomation]----------')
print(train.info())
print('----------[test infomation]----------')
print(test.info())

train data shape:  (156060, 4)
test data shape:  (66292, 3)
----------[train infomation]----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156060 entries, 0 to 156059
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   PhraseId    156060 non-null  int64 
 1   SentenceId  156060 non-null  int64 
 2   Phrase      156060 non-null  object
 3   Sentiment   156060 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 4.8+ MB
None
----------[test infomation]----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66292 entries, 0 to 66291
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   PhraseId    66292 non-null  int64 
 1   SentenceId  66292 non-null  int64 
 2   Phrase      66292 non-null  object
dtypes: int64(2), object(1)
memory usage: 1.5+ MB
None


In [5]:
train_tokens = Sentences2LemmaWords(train.Phrase)
test_tokens = Sentences2LemmaWords(test.Phrase)

2020-07-14 11:02:15,797[DEBUG|preprocessing, 17] Sentence2LemmaWords: start
/content/drive/My Drive/딥러닝 실습/5.4.Sentiment Analysis/preprocessing/text_preprocessing.py:23: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 23 of the file /content/drive/My Drive/딥러닝 실습/5.4.Sentiment Analysis/preprocessing/text_preprocessing.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  sentence = BeautifulSoup(sentence).get_text()


228 / 156060 Phrase

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


10689 / 156060 Phrase

/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'log'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


...

155862 / 156060 Phrase

2020-07-14 11:04:43,810[DEBUG|preprocessing, 36] Sentence2LemmaWords: finish. 계산 시간 148.01412177085876sec
2020-07-14 11:04:43,815[DEBUG|preprocessing, 17] Sentence2LemmaWords: start


156060 / 156060 Phrase
66245 / 66292 Phrase

2020-07-14 11:05:45,695[DEBUG|preprocessing, 36] Sentence2LemmaWords: finish. 계산 시간 61.880117416381836sec


66292 / 66292 Phrase


In [6]:
unique_words, maxlen = GetUnique(train_tokens)
print(len(unique_words), maxlen)

2020-07-14 11:05:45,706[DEBUG|preprocessing, 44] GetUnique: start
2020-07-14 11:05:45,835[DEBUG|preprocessing, 56] GetUnique: finish. 계산 시간 0.12897014617919922sec


13745 48


In [7]:
target = train.Sentiment
y_target = to_categorical(target)
num_classes = y_target.shape[1]

In [8]:
x_train, x_valid, y_train, y_valid = train_test_split(train_tokens, y_target,test_size=0.2, stratify=y_target)

In [9]:
tokenizer = setTokenizer(x_train, len(unique_words))
x_train = LemmaWords2Seqeunces(x_train, tokenizer, maxlen)
x_valid = LemmaWords2Seqeunces(x_valid, tokenizer, maxlen)
x_test = LemmaWords2Seqeunces(test_tokens, tokenizer, maxlen)

2020-07-14 11:05:47,029[DEBUG|preprocessing, 63] setTokenizer: start
2020-07-14 11:05:47,899[DEBUG|preprocessing, 70] setTokenizer: finish. 계산 시간 0.8698043823242188sec
2020-07-14 11:05:47,901[DEBUG|preprocessing, 77] LemmaWords2Seqeunces: start
2020-07-14 11:05:49,060[DEBUG|preprocessing, 84] LemmaWords2Seqeunces: finish. 계산 시간 1.158679723739624sec
2020-07-14 11:05:49,066[DEBUG|preprocessing, 77] LemmaWords2Seqeunces: start
2020-07-14 11:05:49,645[DEBUG|preprocessing, 84] LemmaWords2Seqeunces: finish. 계산 시간 0.5786428451538086sec
2020-07-14 11:05:49,649[DEBUG|preprocessing, 77] LemmaWords2Seqeunces: start
2020-07-14 11:05:50,168[DEBUG|preprocessing, 84] LemmaWords2Seqeunces: finish. 계산 시간 0.5198993682861328sec


In [10]:
x_test.shape

(66292, 48)

In [11]:
model = LSTM_model(len(unique_words), maxlen, num_classes)

logger = Logger('main')
start_time = time.time()
logger.debug('fit start')

model.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=5,batch_size=1024, verbose=1)

fit_time = time.time() - start_time
logger.debug('fit end.fit_time{}'.format(fit_time))

2020-07-14 11:05:50,207[DEBUG|LSTM_model_generate, 9] LSTM_model: start
2020-07-14 11:05:50,995[DEBUG|LSTM_model_generate, 20] LSTM_model: finish
2020-07-14 11:05:50,998[DEBUG|main, 5] fit start
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 124848 samples, validate on 31212 samples
Epoch 1/5
124848/124848 [==============================] - 22s 177us/step - loss: 1.1049 - accuracy: 0.5621 - val_loss: 0.8982 - val_accuracy: 0.6344
Epoch 2/5
124848/124848 [==============================] - 22s 173us/step - loss: 0.8165 - accuracy: 0.6677 - val_loss: 0.8299 - val_accuracy: 0.6629
Epoch 3/5
124848/124848 [==============================] - 21s 171us/step - loss: 0.7351 - accuracy: 0.6981 - val_loss: 0.8157 - val_accuracy: 0.6657
Epoch 4/5
124848/124848 [==============================] - 21s 171us/step - loss: 0.6896 - accuracy: 0.7131 - val_loss: 0.8198 - val_accuracy: 0.6675
Epoch 5/5
124848/124848 [==============================] - 21s 171us/step - loss: 0.6529 - accuracy: 0.7268 - val_loss: 0.8231 - val_accuracy: 0.6667


2020-07-14 11:07:39,765[DEBUG|main, 10] fit end.fit_time108.76765632629395


In [12]:
y_pred = model.predict_classes(x_test)

submission = pd.read_csv('/content/drive/My Drive/딥러닝 실습/5.4.Sentiment Analysis/5.4.data/sampleSubmission.csv')

In [13]:
x_test.shape

(66292, 48)

In [14]:
y_pred.shape

(66292,)

In [15]:
submission.shape

(66292, 2)

In [16]:
submission.to_csv('/content/drive/My Drive/딥러닝 실습/5.4.Sentiment Analysis/submission.csv')